In [10]:
!pip install evaluate
!pip install sacrebleu

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.2 MB/s eta 0:00:00


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 5.6 MB/s eta 0:00:00


# Data Load

In [1]:
from datasets import load_dataset

# text_1 -> ind, text_2 -> mad
mdr = load_dataset("indonlp/NusaX-MT",name="ind-mad") 
mdr

README.md:   0%|          | 0.00/5.64k [00:00<?, ?B/s]

NusaX-MT.py:   0%|          | 0.00/5.69k [00:00<?, ?B/s]

The repository for indonlp/NusaX-MT contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/indonlp/NusaX-MT.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text_1', 'text_2', 'text_1_lang', 'text_2_lang'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['id', 'text_1', 'text_2', 'text_1_lang', 'text_2_lang'],
        num_rows: 100
    })
    test: Dataset({
        features: ['id', 'text_1', 'text_2', 'text_1_lang', 'text_2_lang'],
        num_rows: 400
    })
})

In [2]:
train = mdr["train"]
val = mdr["validation"]
test = mdr["test"]

In [3]:
from transformers.utils import move_cache
move_cache()


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [4]:
from transformers import AutoTokenizer

# https://huggingface.co/indonlp/cendol-mt5-small-inst
checkpoint = "indonlp/cendol-mt5-small-inst" # GANTI MODELNYA
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

In [5]:
prefix = "terjemahkan dari Bahasa Madura ke Bahasa Indonesia: " # nanti pas inference, perlu di append prefix nya!

def preprocess_function(examples):
    # Menambahkan prefix dan melakukan lowercasing pada input
    inputs = [prefix + example.lower() for example in examples["text_2"]]  # optional: .lower() jika perlu
    targets = [example.lower() for example in examples["text_1"]]  # optional: .lower() jika perlu
    
    # Tokenisasi
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    
    return model_inputs

In [6]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_val = val.map(preprocess_function, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [7]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [11]:
import evaluate
import numpy as np

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Post-process texts
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Compute BLEU score
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    # Calculate average generation length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = round(np.mean(prediction_lens), 4)
    
    return result

In [12]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="mad-indo_cendol-mt5-small-inst_25",
    eval_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=28,
    predict_with_generate=True,
    fp16=True,
    generation_max_length=50
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipykernel_23/3685775069.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,1.386802,32.723510,34.650000
2,No log,1.325022,35.067202,34.570000
3,No log,1.307034,36.186774,34.690000
4,No log,1.296602,36.157660,34.580000
5,No log,1.311493,37.060024,34.560000
6,No log,1.304290,37.769231,34.490000
7,No log,1.328349,37.110327,34.420000
8,No log,1.324803,38.478755,34.580000
9,No log,1.339374,37.632639,34.420000
10,No log,1.349649,38.584406,34.310000


TrainOutput(global_step=896, training_loss=0.7953644139426095, metrics={'train_runtime': 580.539, 'train_samples_per_second': 24.116, 'train_steps_per_second': 1.543, 'total_flos': 1783869269606400.0, 'train_loss': 0.7953644139426095, 'epoch': 28.0})

In [16]:
tokenized_test = test.map(preprocess_function, batched=True)

In [15]:
test_results = trainer.evaluate(tokenized_test)
print("Test BLEU score:", test_results["eval_bleu"])

Test BLEU score: 36.53391902933586


In [17]:
test_results

{'eval_loss': 1.1646003723144531,
 'eval_bleu': 36.53391902933586,
 'eval_gen_len': 34.3725,
 'eval_runtime': 27.9181,
 'eval_samples_per_second': 14.328,
 'eval_steps_per_second': 0.895,
 'epoch': 28.0}

## SAVE AND LOAD

In [ ]:
# BUAT NGESAVE KE LOCAL!!!
local_dir = "cendol-mt5-small-inst"

model.save_pretrained(local_dir)
tokenizer.save_pretrained(local_dir)

# NANTI CARA NGE LOAD NYA GINI!!
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# model = AutoModelForSeq2SeqLM.from_pretrained(local_dir)
# tokenizer = AutoTokenizer.from_pretrained(local_dir)

## Inference

In [ ]:
test

In [ ]:
import random
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained(local_dir)
tokenizer = AutoTokenizer.from_pretrained(local_dir)

def translate_and_compare_samples(test_data, num_samples=5):
    samples = random.sample(range(len(test_data['text_1'])), num_samples)
    
    for idx in samples:
        source_text = test_data['text_1'][idx]
        target_text = test_data['text_2'][idx]
        text = f"terjemahkan dari Bahasa Indonesia ke Bahasa Madura: {source_text}"
        inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True)
        outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
        translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"Sampel {idx + 1}")
        print("Source Text (Indonesia):", source_text)
        print("Translated Text (Madura):", translated_text)
        print("Target Text (Expected Madura):", target_text)
        print("=" * 50)

translate_and_compare_samples(test, num_samples=5)


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(local_dir)
tokenizer = AutoTokenizer.from_pretrained(local_dir)

text = "terjemahkan dari Bahasa Indonesia ke Bahasa Madura: gimana kabarmu?"

inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True)
outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)

translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Translated text:", translated_text)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Inisialisasi model dan tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(local_dir)
tokenizer = AutoTokenizer.from_pretrained(local_dir)

# Teks masukan dalam bahasa Madura
text = "terjemahkan dari Bahasa Madura ke Bahasa Indonesia: Semmak bik hotel engkok nginep, pera' ejeleni ajelen soko, ediye bennyak sarah pelean kakananna, kenengngan se leber, ben masenneng"

# Tokenisasi input
inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True)

# Generasi terjemahan
outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)

# Dekode hasil terjemahan
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Translated text:", translated_text)

In [ ]:
# import random
# import pandas as pd
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# import sacrebleu

# # Inisialisasi model dan tokenizer
# model = AutoModelForSeq2SeqLM.from_pretrained(local_dir)
# tokenizer = AutoTokenizer.from_pretrained(local_dir)

# # Fungsi untuk menerjemahkan dan membandingkan hasil dengan skor BLEU
# def translate_and_compare_samples(test_data, num_samples=100):
#     # Ambil beberapa sampel acak dari data test
#     samples = random.sample(range(len(test_data['text_1'])), num_samples)

#     # List untuk menyimpan hasil yang akan dikonversi menjadi tabel
#     results = []

#     for idx in samples:
#         source_text = test_data['text_1'][idx]
#         target_text = test_data['text_2'][idx]
#         text = f"terjemahkan dari Bahasa Indonesia ke Bahasa Madura: {source_text}"
        
#         # Tokenisasi dan terjemahan
#         inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True)
#         outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
        
#         # Dekode hasil terjemahan
#         translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
#         # Hitung skor BLEU
#         bleu = sacrebleu.corpus_bleu([translated_text], [[target_text]]).score
        
#         # Tambahkan hasil ke list
#         results.append({
#             "Source Text (Indonesia)": source_text,
#             "Translated Text (Madura)": translated_text,
#             "Target Text (Expected Madura)": target_text,
#             "BLEU Score": bleu
#         })

#     # Konversi hasil ke dalam DataFrame pandas
#     df_results = pd.DataFrame(results)
#     return df_results

# # Panggil fungsi dan tampilkan hasil dalam bentuk tabel
# df_results = translate_and_compare_samples(test, num_samples=100)
# df_results


In [ ]:
# df_results.to_csv("translation_results_full.csv", index=False)